In [17]:
import glob

exclude = ['./CLAUDE.md','./progress-tracker.md','./project-description.md']
list_of_files = glob.glob('meridian-islands/**/*.md') 
filtered_list = []
for a in list_of_files:
    if a not in exclude:
        filtered_list.append(a)

filtered_list = sorted(filtered_list)

current_list = filtered_list[0]
print(current_list)

markdown_string = " "

with open(current_list, "r", encoding="utf-8") as f:
    converted_text = f.read() 

with open("first.txt", "w", encoding="utf-8") as text_file:
    text_file.write(converted_text)

meridian-islands/01-foundation/world-bible.md


In [51]:
from dataclasses import dataclass

@dataclass
class Chunk:
        text: str
        chunk_id: int
        start_pos: int
        end_pos: int
        word_count: int
        origin_file: str


class Chunker:
    def __init__(self,chunk_size: int,overlap: int):
        self.chunk_size = chunk_size
        self.overlap = overlap
       
    def chunk(self, file_path: str) -> list[Chunk]:
        with open(file_path, "r", encoding="utf-8") as f:
            converted_text = f.read()
            
        chunk_size = self.chunk_size
        overlap = self.overlap
        chunks = []
        i=0
        words = converted_text.split()  

        for start in (range(0, len(words), chunk_size)):
            if i != 0:
                start = start - overlap
            else:
                start = start
            end = min(start+chunk_size, len(words))
            chunk = words[start:end]

            chunks.append(
                Chunk(
                    text=chunk,
                    chunk_id=i,
                    start_pos=start,
                    end_pos=end-1,
                    word_count=len(chunk),
                    origin_file=file_path
                )
            )
            i += 1

        return chunks
    
chunker: Chunker = Chunker(150,25)
chunks: list = chunker.chunk("meridian-islands/01-foundation/world-bible.md")

for c in chunks[:2]:
    print(c,"\n")


Chunk(text=['#', 'Meridian', 'Islands', 'World', 'Bible', '*Master', 'Reference', 'Document', '-', 'All', 'content', 'must', 'align', 'with', 'these', 'established', 'facts*', '##', 'Archipelago', 'Overview', 'The', 'Meridian', 'Islands', 'are', 'a', 'sovereign', 'island', 'nation', 'located', 'in', 'the', 'South', 'Pacific', 'Ocean,', 'approximately', '2,100', 'kilometers', 'east', 'of', 'Australia', 'and', '1,800', 'kilometers', 'north', 'of', 'New', 'Zealand.', 'The', 'archipelago', 'consists', 'of', 'six', 'islands:', 'four', 'main', 'inhabited', 'islands', 'and', 'two', 'smaller', 'islets.', '###', 'The', 'Six', 'Islands', '1.', '**Coral', 'Bay', 'Island**', '(385', 'km²)', '-', 'The', 'capital', 'island', 'and', 'main', 'tourist', 'hub', '2.', '**Azure', 'Peak', 'Island**', '(298', 'km²)', '-', 'Adventure', 'and', 'nature', 'tourism', 'center', '3.', '**Amber', 'Cove', 'Island**', '(156', 'km²)', '-', 'Cultural', 'heartland', 'and', 'traditional', 'crafts', '4.', '**Pearl', 'Shoa